In [1]:
import json
import sys,os
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys

sys.path.extend(['/root/xiaoda/query_topic/'])

In [3]:
import torch
from torch.nn import functional as F
import numpy as np
import random
import torch.nn as nn
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score
from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np

"""
https://github.com/ondrejbohdal/meta-calibration/blob/main/Metrics/metrics.py
"""

class ECE(nn.Module):
    
    def __init__(self, n_bins=15):
        """
        n_bins (int): number of confidence interval bins
        """
        super(ECE, self).__init__()
        bin_boundaries = torch.linspace(0, 1, n_bins + 1)
        self.bin_lowers = bin_boundaries[:-1]
        self.bin_uppers = bin_boundaries[1:]

    def forward(self, logits, labels, mode='logits'):
        if mode == 'logits':
            softmaxes = F.softmax(logits, dim=1)
        else:
            softmaxes = logits
        # softmaxes = F.softmax(logits, dim=1)
        confidences, predictions = torch.max(softmaxes, 1)
        accuracies = predictions.eq(labels)
        
        ece = torch.zeros(1, device=logits.device)
        for bin_lower, bin_upper in zip(self.bin_lowers, self.bin_uppers):
            # Calculated |confidence - accuracy| in each bin
            in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
            prop_in_bin = in_bin.float().mean()
            if prop_in_bin.item() > 0:
                accuracy_in_bin = accuracies[in_bin].float().mean()
                avg_confidence_in_bin = confidences[in_bin].mean()
                ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin

        return ece

In [4]:
import torch
import json
import sys
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizerFast
import transformers
from datetime import timedelta

import os, sys

from nets.them_classifier import MyBaseModel, RobertaClassifier

import configparser
from tqdm import tqdm

cur_dir_path = '/root/xiaoda/query_topic/'

def load_label(filepath):
    label_list = []
    with open(filepath, 'r') as frobj:
        for line in frobj:
            label_list.append(line.strip())
        n_classes = len(label_list)

        label2id = {}
        id2label = {}
        for idx, label in enumerate(label_list):
            label2id[label] = idx
            id2label[idx] = label
        return label2id, id2label

class RiskInfer(object):
    def __init__(self, config_path):

        import torch, os, sys

        con = configparser.ConfigParser()
        con_path = os.path.join(cur_dir_path, config_path)
        con.read(con_path, encoding='utf8')

        args_path = dict(dict(con.items('paths')), **dict(con.items("para")))
        self.tokenizer = BertTokenizerFast.from_pretrained(args_path["model_path"], do_lower_case=True)

        from collections import OrderedDict
        self.schema_dict = OrderedDict({})

        for label_index, schema_info in enumerate(args_path["label_path"].split(',')):
            schema_type, schema_path = schema_info.split(':')
            schema_path = os.path.join(cur_dir_path, schema_path)
            print(schema_type, schema_path, '===schema-path===')
            label2id, id2label = load_label(schema_path)
            self.schema_dict[schema_type] = {
                'label2id':label2id,
                'id2label':id2label,
                'label_index':label_index
            }
            print(self.schema_dict[schema_type], '==schema_type==', schema_type)
        
        output_path = os.path.join(cur_dir_path, args_path['output_path'])

        from roformer import RoFormerModel, RoFormerConfig

        config = RoFormerConfig.from_pretrained(args_path["model_path"])
        encoder = RoFormerModel(config=config)
        
        encoder_net = MyBaseModel(encoder, config)

        self.device = "cuda:3" if torch.cuda.is_available() else "cpu"

        classifier_list = []

        schema_list = list(self.schema_dict.keys())

        for schema_key in schema_list:
            classifier = RobertaClassifier(
                hidden_size=config.hidden_size, 
                dropout_prob=con.getfloat('para', 'out_dropout_rate'),
                num_labels=len(self.schema_dict[schema_key]['label2id']), 
                dropout_type=con.get('para', 'dropout_type'))
            classifier_list.append(classifier)

        classifier_list = nn.ModuleList(classifier_list)

        class MultitaskClassifier(nn.Module):
            def __init__(self, transformer, classifier_list):
                super().__init__()

                self.transformer = transformer
                self.classifier_list = classifier_list

            def forward(self, input_ids, input_mask, 
                        segment_ids=None, 
                        transformer_mode='mean_pooling', 
                        dt_idx=None):
                hidden_states = self.transformer(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              return_mode=transformer_mode)
                outputs_list = []
                
                for idx, classifier in enumerate(self.classifier_list):
                    
                    if dt_idx is not None and idx != dt_idx:
                        continue
                    
                    ce_logits = classifier(hidden_states)
                    outputs_list.append(ce_logits)
                return outputs_list, hidden_states

        self.net = MultitaskClassifier(encoder_net, classifier_list).to(self.device)

        # eo = 9
        # ckpt = torch.load(os.path.join(output_path, 'multitask_cls.pth.{}.raw'.format(eo)), map_location=self.device)
        # # ckpt = torch.load(os.path.join(output_path, 'multitask_cls.pth.{}.raw.focal'.format(eo)), map_location=self.device)
        # # ckpt = torch.load(os.path.join(output_path, 'multitask_contrast_cls.pth.{}'.format(eo)), map_location=self.device)
        # self.net.load_state_dict(ckpt)
        # self.net.eval()
        
    def reload(self, model_path):
        ckpt = torch.load(model_path, map_location=self.device)
        self.net.load_state_dict(ckpt)
        self.net.eval()

    def predict(self, text):

        """抽取输入text所包含的类型
        """
        encoder_txt = self.tokenizer.encode_plus(text, max_length=256)
        input_ids = torch.tensor(encoder_txt["input_ids"]).long().unsqueeze(0).to(self.device)
        token_type_ids = torch.tensor(encoder_txt["token_type_ids"]).unsqueeze(0).to(self.device)
        attention_mask = torch.tensor(encoder_txt["attention_mask"]).unsqueeze(0).to(self.device)
        
        scores_dict = {}
        with torch.no_grad():
            [logits_list, 
            hidden_states] = self.net(input_ids, 
                attention_mask, token_type_ids, transformer_mode='cls')
        for schema_type, logits in zip(list(self.schema_dict.keys()), logits_list):
            scores = torch.nn.Softmax(dim=1)(logits)[0].data.cpu().numpy()
            scores_dict[schema_type] = []
            for index, score in enumerate(scores):
                scores_dict[schema_type].append([self.schema_dict[schema_type]['id2label'][index], 
                                        float(score)])
        return scores_dict
    
    def predict_batch(self, text):
        if isinstance(text, list):
            text_list = text
        else:
            text_list = [text]
        model_input = self.tokenizer(text_list, return_tensors="pt",padding=True)
        for key in model_input:
            model_input[key] = model_input[key].to(self.device)
        with torch.no_grad():
            [logits_list, 
            hidden_states] = self.net(model_input['input_ids'], 
                model_input['attention_mask'], 
                model_input['token_type_ids'], transformer_mode='cls')
        score_dict_list = []
        for idx, text in enumerate(text_list):
            scores_dict = {}
            for schema_type, logits in zip(list(self.schema_dict.keys()), logits_list):
                scores = torch.nn.Softmax(dim=1)(logits)[idx].data.cpu().numpy()
                scores_dict[schema_type] = []
                for index, score in enumerate(scores):
                    scores_dict[schema_type].append([self.schema_dict[schema_type]['id2label'][index], 
                                            float(score)])
            score_dict_list.append(scores_dict)
        return score_dict_list

# risk_api = RiskInfer('./risk_data/config.ini')
risk_api = RiskInfer('./risk_data_v5/config.ini')




senti_query /root/xiaoda/query_topic/risk_data_v5/senti_query_label.txt ===schema-path===
{'label2id': {'负向': 0, '中性': 1, '正向': 2}, 'id2label': {0: '负向', 1: '中性', 2: '正向'}, 'label_index': 0} ==schema_type== senti_query
senti /root/xiaoda/query_topic/risk_data_v5/senti_label.txt ===schema-path===
{'label2id': {'负向': 0, '正向': 1}, 'id2label': {0: '负向', 1: '正向'}, 'label_index': 1} ==schema_type== senti
bias /root/xiaoda/query_topic/risk_data_v5/bias_label.txt ===schema-path===
{'label2id': {'偏见': 0, '正常': 1}, 'id2label': {0: '偏见', 1: '正常'}, 'label_index': 2} ==schema_type== bias
ciron /root/xiaoda/query_topic/risk_data_v5/ciron_label.txt ===schema-path===
{'label2id': {'讽刺': 0, '正常': 1}, 'id2label': {0: '讽刺', 1: '正常'}, 'label_index': 3} ==schema_type== ciron
intent /root/xiaoda/query_topic/risk_data_v5/intention_label_v0.txt ===schema-path===
{'label2id': {'主观评价/比较/判断': 0, '寻求建议/帮助': 1, '其它': 2}, 'id2label': {0: '主观评价/比较/判断', 1: '寻求建议/帮助', 2: '其它'}, 'label_index': 4} ==schema_type== intent

12/27/2022 08:18:06 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
12/27/2022 08:18:06 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
12/27/2022 08:18:06 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
12/27/2022 08:18:06 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
12/27/2022 08:18:06 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
12/27/2022 08:18:06 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
12/27/2022 08:18:06 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
12/27/2022 08:18:06 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++


In [5]:
import csv
l = 0
reader = csv.reader(open('/data/albert.xht/raw_chat_corpus/model_risk_xiaoda/Query风险分类_全部数据.csv'), delimiter="\t", quotechar=None)
for idx, item in enumerate(reader):
    # print(item)
    l += 1
l

9965

In [ ]:
with open("/data/albert.xht/pretrained_model_risk/corpus/efaqa-corpus-zh/efaqa-corpus-zh.utf8", "r") as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        title = ''.join(re.split('[\s,]', content['title'])[1:])
        if len(title) >= 5:
            if  s3_mapping[content['label']['s3']] in ['正在进行的自杀行为', '策划进行的自杀行为', '自残']:
                tmp = {
                    'title':title,
                    'label':['风险']
                }
                print(tmp)

In [33]:
# risk_api.reload('/data/albert.xht/xiaodao/risk_classification/multitask_raw_filter_senti_query_risk_v5_intent_v2_3/multitask_cls.pth.4')
# risk_api.reload('/data/albert.xht/xiaodao/risk_classification/multitask_raw_filter_senti_query_risk_v5_intent_v2-1_3//multitask_cls.pth.4')

# risk_api.reload('/data/albert.xht/xiaodao/risk_classification/multitask_raw_filter_senti_query_risk_v6_intent_v2-1_10/multitask_cls.pth.6')

risk_api.reload('/data/albert.xht/xiaodao/risk_classification/multitask_raw_filter_senti_query_risk_v6_intent_v2-1_10_no_symbol/multitask_cls.pth.5')


# risk_api.reload('/data/albert.xht/xiaodao/risk_classification/multitask_raw_filter_senti_query_risk_v6_no_offensive_intent_v2-1_10//multitask_cls.pth.6')
# risk_api.reload('/data/albert.xht/xiaodao/risk_classification/multitask_raw_filter_senti_query_risk_v5_no_offensive_intent_v2-1_3/multitask_cls.pth.4')


# risk_api.reload('/data/albert.xht/xiaodao/risk_classification/multitask_raw_filter_senti_query_risk_v4_intent_v2/multitask_cls.pth.9')


In [17]:
def batch_inference(input_path, output_path):
    from tqdm import tqdm
    import numpy as np
    import json, re

    def risk_predict_batch(text):
        if isinstance(text, list):
            text_list = text
        else:
            text_list = [text]
        result_list = risk_api.predict_batch(text_list)
        return result_list
    
    print(input_path, '===input-path===')
    print(output_path, '===output-path===')
    
    with open(output_path, 'w') as fwobj:
        with open(input_path, 'r') as frobj:
            queue = []
            t = []
            for line in tqdm(frobj):
                content = json.loads(line.strip())
                content['text'] = re.sub('请问', '', content['text'])
                text = re.sub(r"([，\_《。》、？；：‘’＂“”【「】」·！@￥…（）—\,\<\.\>\/\?\;\:\'\"\[\]\{\}\~\`\!\@\#\$\%\^\&\*\(\)\-\=\+])+", "", content['text'])   # 合并正文中过多的空格
                queue.append(text)
                t.append(content)
                if np.mod(len(queue), 512) == 0:
                    probs = risk_predict_batch(queue)
                    for prob_dict, text, tt in zip(probs, queue, t):
                        content = {
                            'text':tt['text'],
                            'topic':tt['label'],
                            'score_list':prob_dict,
                            # 'score_list': tt['score_list']
                        }
                        fwobj.write(json.dumps(content, ensure_ascii=False)+'\n')
                    queue = []
                    t = []
            if queue:
                probs = risk_predict_batch(queue)
                for prob_dict, text, tt in zip(probs, queue, t):
                    content = {
                        'text':tt['text'],
                        'topic':tt['label'],
                        'score_list':prob_dict,
                        # 'score_list': tt['score_list']
                    }
                    fwobj.write(json.dumps(content, ensure_ascii=False)+'\n')
                    



In [18]:
input_path = '/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json'
output_path = '/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.all_risk.v6'


batch_inference(input_path, output_path)


0it [00:00, ?it/s]

/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json ===input-path===
/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.all_risk.v6 ===output-path===


2071401it [29:41, 1162.82it/s]


In [23]:
black = []
white = []

with open(output_path) as frobj:
    for line in tqdm(frobj):
        content = json.loads(line.strip())
        if content['score_list']['query_risk'][0][1] > 0.9:
            black.append(content)
        else:
            white.append(content)

2071401it [01:29, 23231.96it/s]


In [46]:
black[150:160]

[{'text': '现代医学已经发展到了什么恐怖的水平？',
  'topic': ['健康'],
  'score_list': {'senti_query': [['负向', 0.7500477433204651],
    ['中性', 0.2268967479467392],
    ['正向', 0.02305549569427967]],
   'senti': [['负向', 0.9985207915306091], ['正向', 0.0014792227884754539]],
   'bias': [['偏见', 0.0899629145860672], ['正常', 0.910037100315094]],
   'ciron': [['讽刺', 0.13127018511295319], ['正常', 0.868729829788208]],
   'intent': [['主观评价/比较/判断', 0.11349240690469742],
    ['寻求建议/帮助', 0.001357473200187087],
    ['其它', 0.8851500749588013]],
   'offensive': [['冒犯', 0.9177260398864746], ['正常', 0.08227396011352539]],
   'query_risk': [['风险', 0.9998883008956909],
    ['个人信息', 4.695604275184451e-06],
    ['正常', 0.00010706200555432588]],
   'teenager': [['不良', 0.27327635884284973], ['正常', 0.7267236113548279]]}},
 {'text': '同样作为亚洲经济强国的韩国和日本，为什么在文化输出方面能力上后者远远强于前者？',
  'topic': ['国家'],
  'score_list': {'senti_query': [['负向', 0.0009883948368951678],
    ['中性', 0.02751978300511837],
    ['正向', 0.9714917540550232]],
   'senti': [['

In [70]:
white_dict = {}
from collections import Counter
t = Counter()
for d in white:
    t[d['topic'][0]] += 1
    if d['topic'][0] not in white_dict:
        white_dict[d['topic'][0]] = []
    white_dict[d['topic'][0]].append(d)

In [37]:
risk_api.predict('美国的参议员为什么不交给众议院来选举')

{'senti_query': [['负向', 0.9854410290718079],
  ['中性', 0.014096961356699467],
  ['正向', 0.0004620045074261725]],
 'senti': [['负向', 0.9976153373718262], ['正向', 0.002384644001722336]],
 'bias': [['偏见', 0.03199134021997452], ['正常', 0.9680086374282837]],
 'ciron': [['讽刺', 0.26645970344543457], ['正常', 0.7335402369499207]],
 'intent': [['主观评价/比较/判断', 0.05716642737388611],
  ['寻求建议/帮助', 0.011795553378760815],
  ['其它', 0.9310380220413208]],
 'offensive': [['冒犯', 0.6193205714225769], ['正常', 0.3806794583797455]],
 'query_risk': [['风险', 0.9945979118347168],
  ['个人信息', 6.596983439521864e-05],
  ['正常', 0.00533616216853261]],
 'teenager': [['不良', 0.10363443195819855], ['正常', 0.8963655233383179]]}

In [275]:
import re
re.split('(\\[unused\d+\\])', '[unused1]后的数据规范环境[unused2]gsdhkjsgaf[unused2][SEP]')

['', '[unused1]', '后的数据规范环境', '[unused2]', 'gsdhkjsgaf', '[unused2]', '[SEP]']

In [331]:
risk_api.tokenizer('', add_special_tokens=False)

{'input_ids': [], 'token_type_ids': [], 'attention_mask': []}

In [268]:
from tqdm import tqdm
import numpy as np
import json, re

def risk_predict_batch(text):
    if isinstance(text, list):
        text_list = text
    else:
        text_list = [text]
    result_list = risk_api.predict_batch(text_list)
    return result_list

# with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.all_risk', 'w') as fwobj:
with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.all_risk_v5', 'w') as fwobj:
    with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json', 'r') as frobj:
        queue = []
        t = []
        for line in tqdm(frobj):
            content = json.loads(line.strip())
            content['text'] = re.sub('请问', '', content['text'])
            queue.append(content['text'])
            t.append(content)
            if np.mod(len(queue), 512) == 0:
                probs = risk_predict_batch(queue)
                for prob_dict, text, tt in zip(probs, queue, t):
                    content = {
                        'text':text,
                        'topic':tt['label'],
                        'score_list':prob_dict
                    }
                    fwobj.write(json.dumps(content, ensure_ascii=False)+'\n')
                queue = []
                t = []
        if queue:
            probs = risk_predict_batch(queue)
            for prob_dict, text, tt in zip(probs, queue, t):
                content = {
                    'text':text,
                    'topic':tt['label'],
                    'score_list':prob_dict
                }
                fwobj.write(json.dumps(content, ensure_ascii=False)+'\n')
                    

2071401it [30:34, 1129.22it/s]


In [76]:
from tqdm import tqdm
import numpy as np
import json, re

def risk_predict_batch(text):
    if isinstance(text, list):
        text_list = text
    else:
        text_list = [text]
    result_list = risk_api.predict_batch(text_list)
    return result_list

offensive = []
with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.offensive.all', 'r') as frobj:
    queue = []
    t = []
    for line in tqdm(frobj):
        content = json.loads(line.strip())
        content['text'] = re.sub('请问', '', content['text'])
        queue.append(content['text'])
        t.append(content)
        if np.mod(len(queue), 512) == 0:
            probs = risk_predict_batch(queue)
            for prob_dict, text, tt in zip(probs, queue, t):
                content = {
                    'text':text,
                    'topic':tt['label'],
                    'score_list':prob_dict
                }
                offensive.append(content)
            queue = []
            t = []
    if queue:
        probs = risk_predict_batch(queue)
        for prob_dict, text, tt in zip(probs, queue, t):
            content = {
                'text':text,
                'topic':tt['label'],
                'score_list':prob_dict
            }
            offensive.append(content)


15414it [00:14, 1069.51it/s]


In [ ]:
with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.white', 'w') as fwobj:
    with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json', 'r') as frobj:
        data_dict = {}
        for line in frobj:
            content = json.loads(line.strip())
            if content['text'] not in data_dict:
                data_dict[content['text']] = []
    
    

In [7]:
from tqdm import tqdm
import numpy as np
import json, re

def risk_predict_batch(text):
    if isinstance(text, list):
        text_list = text
    else:
        text_list = [text]
    result_list = risk_api.predict_batch(text_list)
    return result_list

In [10]:
risk_api.device

'cuda:3'

In [ ]:
from tqdm import tqdm
import numpy as np
import json, re

def risk_predict_batch(text):
    if isinstance(text, list):
        text_list = text
    else:
        text_list = [text]
    result_list = risk_api.predict_batch(text_list)
    return result_list

with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.bias_ciron', 'w') as fwobj:
    with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json', 'r') as frobj:
        for line in frobj:
            content = json.loads(line.strip())
            if content['label'][0] not in data_dict:
                data_dict[content['label'][0]] = []
            data_dict[content['label'][0]].append(content)

    train_sample = []
    import random
    for key in data_dict:
        random.shuffle(data_dict[key])
        train_sample.extend(data_dict[key][:int(0.2*len(data_dict[key]))])
    cnt = 1
    queue = []
    for content in tqdm(train_sample):
        content['text'] = re.sub('请问', '', content['text'])
        queue.append(content['text'])
        if np.mod(len(queue), 256) == 0:
            probs = risk_predict_batch(queue)
            for prob_dict, text in zip(probs, queue):
                score_list = []
                for key in ['bias', 'ciron', 'teenager']:
                    if prob_dict[key][0][1] > 0.9:
                        score_list.append([key, float(prob_dict[key][0][1])])
                key = 'query_risk'
                if prob_dict[key][0][1] > 0.9:
                    score_list.append([prob_dict[key][0][0], float(prob_dict[key][0][1])])
                if prob_dict[key][1][1] > 0.9:
                    score_list.append([prob_dict[key][1][0], float(prob_dict[key][1][1])])
                if score_list:
                    content = {
                        'text':text,
                        'label':['风险'],
                        'score_list':score_list
                    }
                    fwobj.write(json.dumps(content, ensure_ascii=False)+'\n')
            queue = []
    if queue:
        probs = risk_predict_batch(queue)
        for prob_dict, text in zip(probs, queue):
            score_list = []
            for key in ['bias', 'ciron', 'teenager']:
                if prob_dict[key][0][1] > 0.9:
                    score_list.append([key, float(prob_dict[key][0][1])])
            key = 'query_risk'
            if prob_dict[key][0][1] > 0.9:
                score_list.append([prob_dict[key][0][0], float(prob_dict[key][0][1])])
            if prob_dict[key][1][1] > 0.9:
                score_list.append([prob_dict[key][1][0], float(prob_dict[key][1][1])])

            if score_list:
                content = {
                    'text':text,
                    'label':['风险'],
                    'score_list':score_list
                }
                fwobj.write(json.dumps(content, ensure_ascii=False)+'\n')

In [107]:
prob_dict[key][0]

['偏见', 0.4076650142669678]

In [41]:
# risk_api.reload('/data/albert.xht/xiaodao/risk_classification/multitask_raw_all_ce_256/multitask_cls.pth.9')

# risk_api.reload('/data/albert.xht/xiaodao/risk_classification/multitask_raw_all_intent_v1/multitask_cls.pth.9')

# risk_api.reload('/data/albert.xht/xiaodao/risk_classification/multitask_mtdnn_ce_256/multitask_cls.pth.9')
# risk_api.reload('/data/albert.xht/xiaodao/risk_classification/multitask_contrast_intent_v1/multitask_cls.pth.9')

# risk_api.reload('/data/albert.xht/xiaodao/risk_classification/multitask_raw_all_ce_256_20/multitask_cls.pth.19')


# risk_api.reload('/data/albert.xht/xiaodao/query_risk_v3/multitask_raw_filter_senti_query_risk_v3/multitask_cls.pth.9')


In [367]:
model_path = [
'/data/albert.xht/xiaodao/risk_classification/multitask_raw_all_ce_256/multitask_cls.pth.9',
'/data/albert.xht/xiaodao/risk_classification/multitask_raw_all_intent_v1/multitask_cls.pth.9',
'/data/albert.xht/xiaodao/risk_classification/multitask_mtdnn_ce_256/multitask_cls.pth.9',
'/data/albert.xht/xiaodao/risk_classification/multitask_raw_all_ce_256_20/multitask_cls.pth.19',
]

from tqdm import tqdm
total_result = []
for model in model_path:
    result_list = []
    risk_api.reload(model)
    with open('/data/albert.xht/raw_chat_corpus/model_risk_xiaoda/疑似有风险query_from对话预训练数据-20221130.txt') as frobj:
        for line in tqdm(frobj):
            text = line.strip()
            result = risk_api.predict(text)
            result_list.append((text, result))
    total_result.append(result_list)

result_matrix = []
for i in range(len(total_result[0])):
    p = []
    for tmp in total_result:
        item = tmp[i]
        for key in ['senti', 'bias', 'ciron', 'offensive', 'teenager', 'query_risk']:
            if item[1][key][0][1] > 0.6:
                p.append(1)
            else:
                p.append(0)
    result_matrix.append(p)



56443it [07:27, 126.13it/s]
56443it [07:29, 125.59it/s]
56443it [07:25, 126.73it/s]
56443it [07:25, 126.60it/s]


array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1])

In [176]:
from tqdm import tqdm
result_list = []
with open('/data/albert.xht/raw_chat_corpus/model_risk_xiaoda/疑似有风险query_from对话预训练数据-20221130.txt') as frobj:
    for line in tqdm(frobj):
        text = line.strip()
        result = risk_api.predict(text)
        result_list.append((text, result))
        

56443it [08:00, 117.48it/s]


In [362]:
result_matrix = []
for item in result_list:
    p = []
    for key in ['senti', 'bias', 'ciron', 'offensive', 'teenager']:
        if item[1][key][0][1] > 0.59:
            p.append(1)
        else:
            p.append(0)
    result_matrix.append(p)
        
        
# result_matrix = []
# for item in result_list:
#     p = []
#     for key in ['senti', 'bias', 'ciron', 'offensive', 'teenager', 'query_risk']:
#         if item[1][key][0][1] >= 0.6:
#             p.append(1)
#         else:
#             p.append(0)
#     result_matrix.append(p)
    

In [420]:
result_matrix = np.array(result_matrix)
votes = np.sum(result_matrix, axis=-1)
labels = np.array(votes >= 6).astype(np.int)
sum(labels)

36067

In [463]:

from sklearn.ensemble import RandomForestClassifier
result_matrix = np.array(result_matrix)

def SIMPLE(result_matrix):
    votes = np.sum(result_matrix, axis=-1)
    labels = np.array(votes >= 6).astype(np.int)
    clf = RandomForestClassifier(max_depth=2, random_state=0, ccp_alpha=0.1)
    
    for i in range(5):
        result = clf.fit(result_matrix, labels)
        probs = clf.predict_proba(result_matrix)
        print(probs.shape)
        labels = np.argmax(probs, axis=-1)
    return probs


probs = SIMPLE(result_matrix)
        

(56443, 2)
(56443, 2)
(56443, 2)
(56443, 2)
(56443, 2)


In [464]:
ff = []
clean = []
for idx in range(probs.shape[0]):
    if probs[idx,1] >= 0.8:
        ff.append((idx, result_list[idx], probs[idx]))
    else:
        clean.append((idx, result_list[idx], probs[idx]))
        


In [23]:
with open('/data/albert.xht/raw_chat_corpus/model_risk_xiaoda/query_risk_corpus.json.risk', 'w') as fwobj:
    with open('/data/albert.xht/raw_chat_corpus/model_risk_xiaoda/query_risk_corpus.json', 'r') as frobj:
        queue = []
        t = []
        for line in tqdm(frobj):
            content = json.loads(line.strip())
            # content['text'] = re.sub('请问', '', content['text'])
            queue.append(content['text'])
            t.append(content)
            if np.mod(len(queue), 512) == 0:
                probs = risk_predict_batch(queue)
                for prob_dict, text, tt in zip(probs, queue, t):
                    content = {
                        'text':text,
                        'topic':tt['label'],
                        'score_list':prob_dict
                    }
                    fwobj.write(json.dumps(content, ensure_ascii=False)+'\n')
                queue = []
                t = []
        if queue:
            probs = risk_predict_batch(queue)
            for prob_dict, text, tt in zip(probs, queue, t):
                content = {
                    'text':text,
                    'topic':tt['label'],
                    'score_list':prob_dict
                }
                fwobj.write(json.dumps(content, ensure_ascii=False)+'\n')

24721it [00:22, 1076.00it/s]


In [480]:
with open('/data/albert.xht/raw_chat_corpus/model_risk_xiaoda/query_risk_corpus.json', 'w') as fwobj:
    for item in ff:
        tmp = {
            'text':item[1][0],
            'label':['风险']
        }
        fwobj.write(json.dumps(tmp, ensure_ascii=False)+'\n')


In [221]:
candidate = [] 
positive = []
for item in result_list:
    if item[1]['senti'][0][1] >= 0.7 or item[1]['bias'][0][1] >= 0.8 or item[1]['ciron'][0][1] >= 0.8 or item[1]['offensive'][0][1] >= 0.8:
        candidate.append(item)
    else:
        positive.append(item)

In [10]:

offensive = []
with open('/data/albert.xht/sentiment/dev/offensive_cold.json') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        offensive.append(content)
        
offensive_test = []
with open('/data/albert.xht/sentiment/test/offensive_cold.json') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        offensive_test.append(content)

        
cdia_bias = []
with open('/data/albert.xht/sentiment/dev/cdial_bias.json') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        cdia_bias.append(content)
        
senti_copr = []
with open('/data/albert.xht/sentiment/dev/senti_copr.json') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        senti_copr.append(content)
        
ciron = []
with open('/data/albert.xht/sentiment/dev/chinese_ciron.json') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        ciron.append(content)

senti_smp = []
with open('/data/albert.xht/sentiment/dev/senti_smp_usual.json') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        senti_smp.append(content)
        
senti_smpecisa = []
with open('/data/albert.xht/sentiment/dev/senti_smpecisa.json') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        senti_smpecisa.append(content)

        
senti_query = []
with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_valid.json.filter.0.7') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        senti_query.append(content)

In [15]:
from sklearn.metrics import classification_report
from tqdm import tqdm
import re

def eval_all(data, model, key):
    pred = []
    gold = []
    pred_score = []
    for item in tqdm(data):
        gold.append(item['label'][0])
        if isinstance(item['text'], list):
            text = "\n".join(item['text'])
        else:
            text = item['text']
        text = re.sub(r"([，\_《。》、？；：‘’＂“”【「】」·！@￥…（）—\,\<\.\>\/\?\;\:\'\"\[\]\{\}\~\`\!\@\#\$\%\^\&\*\(\)\-\=\+])+", "", text)   # 合并正文中过多的空格

        result = model.predict(text)
        score = sorted(result[key], key=lambda u:u[1], reverse=True)
        pred.append(score[0][0])
        pred_score.append(result[key])
    print(classification_report(gold, pred, digits=4))
    return pred, gold, pred_score
    


In [12]:

def evaluation_ece(pred_score, gold):
    pred_score_l = []
    mapping_dict = {}
    for item in pred_score:
        pred_score_l.append([])
        for idx, p in enumerate(item):
            if p[0] not in mapping_dict:
                mapping_dict[p[0]] = idx
            pred_score_l[-1].append(p[1])
    pred_score_l = torch.tensor(pred_score_l)
    gold_l = torch.tensor([mapping_dict[item] for item in gold])

    ece_fn = ECE(n_bins=15)
    print(ece_fn(pred_score_l, gold_l, mode='probs'), '==ece==')
# pred, gold, pred_score = eval_all(offensive_test, risk_api, 'offensive')
# evaluation_ece(pred_score, gold)


{'冒犯': 0, '正常': 1}
tensor([0.1119]) ==ece==


In [40]:
p

['正常', 0.9768216013908386]

In [37]:
mapping_dict

{}

In [28]:
item

'正常'

In [13]:
def evaluation(model_path):
    risk_api.reload(model_path)
    print('===offensive===')
    pred, gold, pred_score = eval_all(offensive_test, risk_api, 'offensive')
    evaluation_ece(pred_score, gold)
    print('===cdia-bias===')
    pred, gold, pred_score = eval_all(cdia_bias, risk_api, 'bias')
    evaluation_ece(pred_score, gold)
    print('===ciron===')
    pred, gold, pred_score = eval_all(ciron, risk_api, 'ciron')
    evaluation_ece(pred_score, gold)
    print('===chsenti===')
    pred, gold, pred_score = eval_all(senti_copr, risk_api, 'senti')
    evaluation_ece(pred_score, gold)
    print('===senti_smpecisa===')
    pred, gold, pred_score = eval_all(senti_smpecisa, risk_api, 'senti')
    evaluation_ece(pred_score, gold)
    print('===senti_smp===')
    pred, gold, pred_score = eval_all(senti_smp, risk_api, 'senti')
    evaluation_ece(pred_score, gold)
    print('===senti_query===')
    pred, gold, pred_score = eval_all(senti_query, risk_api, 'senti')
    evaluation_ece(pred_score, gold)
    

In [6]:
risk_api.reload('/data/albert.xht/xiaodao/risk_classification/multitask_raw_filter_senti_query_risk_v4_intent_v2/multitask_cls.pth.9')

# risk_api.reload('/data/albert.xht/xiaodao/risk_classification/multitask_query_risk_20221222/multitask_cls.pth.9')

risk_api.reload('/data/albert.xht/xiaodao/risk_classification/multitask_raw_filter_senti_query_risk_v4_intent_v2_5_aug/multitask_cls.pth.4')

intent_v2 = []
with open('/data/albert.xht/raw_chat_corpus/xiaoda/intention_data_v2/dev.txt') as frobj:
    for line in frobj:
        intent_v2.append(json.loads(line.strip()))
pred, gold, pred_score = eval_all(intent_v2, risk_api, 'intent')
evaluation_ece(pred_score, gold)

NameError: name 'eval_all' is not defined

In [16]:
intent_v2 = []
with open('/data/albert.xht/raw_chat_corpus/xiaoda/intention_data_v2/dev.txt') as frobj:
    for line in frobj:
        intent_v2.append(json.loads(line.strip()))
pred, gold, pred_score = eval_all(intent_v2, risk_api, 'intent')
evaluation_ece(pred_score, gold)

100%|██████████| 3000/3000 [00:25<00:00, 118.52it/s]

              precision    recall  f1-score   support

  主观评价/比较/判断     0.9307    0.9801    0.9548       603
          其它     0.9922    0.9734    0.9827      2219
     寻求建议/帮助     0.9043    0.9551    0.9290       178

    accuracy                         0.9737      3000
   macro avg     0.9424    0.9695    0.9555      3000
weighted avg     0.9746    0.9737    0.9739      3000

tensor([0.0159]) ==ece==


In [18]:
risk_api.predict('外卖骑手来不及送餐赶紧闯红灯')


{'senti_query': [['负向', 0.39223232865333557],
  ['中性', 0.4583439230918884],
  ['正向', 0.149423748254776]],
 'senti': [['负向', 0.700639545917511], ['正向', 0.2993604242801666]],
 'bias': [['偏见', 0.19105122983455658], ['正常', 0.808948814868927]],
 'ciron': [['讽刺', 0.07068543881177902], ['正常', 0.9293144941329956]],
 'intent': [['主观评价/比较/判断', 0.00011481617548270151],
  ['寻求建议/帮助', 0.0017586412141099572],
  ['其它', 0.9981265664100647]],
 'offensive': [['冒犯', 0.05265172943472862], ['正常', 0.9473482966423035]],
 'query_risk': [['风险', 0.026251530274748802],
  ['个人信息', 6.0953414504183456e-05],
  ['正常', 0.9736875295639038]],
 'teenager': [['不良', 0.06425187736749649], ['正常', 0.9357481598854065]]}

In [ ]:
# evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_raw_filter_senti_query_risk_v5_intent_v2_3/multitask_cls.pth.4')


In [21]:
evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_raw_filter_senti_query_risk_v6_intent_v2-1_10_no_symbol/multitask_cls.pth.9')


  0%|          | 12/5304 [00:00<00:45, 115.43it/s]

===offensive===


  0%|          | 12/2829 [00:00<00:23, 118.02it/s]

              precision    recall  f1-score   support

          冒犯     0.7299    0.8557    0.7878      2106
          正常     0.8928    0.7914    0.8391      3198

    accuracy                         0.8169      5304
   macro avg     0.8113    0.8235    0.8134      5304
weighted avg     0.8281    0.8169    0.8187      5304

tensor([0.1097]) ==ece==
===cdia-bias===


  1%|▏         | 12/875 [00:00<00:07, 118.77it/s]

...

              precision    recall  f1-score   support

          正向     0.7885    0.8226    0.8052      1201
          负向     0.8331    0.8005    0.8164      1328

    accuracy                         0.8110      2529
   macro avg     0.8108    0.8115    0.8108      2529
weighted avg     0.8119    0.8110    0.8111      2529

tensor([0.1149]) ==ece==
===senti_smp===


  0%|          | 12/50509 [00:00<07:03, 119.23it/s]

              precision    recall  f1-score   support

          正向     0.8116    0.9067    0.8565      1126
          负向     0.9338    0.8620    0.8965      1718

    accuracy                         0.8797      2844
   macro avg     0.8727    0.8844    0.8765      2844
weighted avg     0.8854    0.8797    0.8807      2844

tensor([0.0631]) ==ece==
===senti_query===


 17%|█▋        | 8365/50509 [01:10<05:55, 118.62it/s]


KeyboardInterrupt: 

In [19]:
# evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_raw_filter_senti_query_risk_v5_no_offensive_intent_v2-1_3/multitask_cls.pth.4')


In [20]:
# evaluation('/data/albert.xht/xiaodao/multitask_raw_filter_senti_query_risk_v5_intent_v2-1_3/multitask_cls.pth.4')


In [1]:
import random
random.randint(0, 1), random.sample(list(range(1, 2)), k=0), range(1, 0)

(0, [], range(1, 0))

In [23]:
evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_raw_filter_senti_query_risk_v4_intent_v2_5_aug/multitask_cls.pth.4')


===offensive===


100%|██████████| 5304/5304 [00:43<00:00, 121.00it/s]


              precision    recall  f1-score   support

          冒犯     0.7306    0.8523    0.7868      2106
          正常     0.8908    0.7930    0.8390      3198

    accuracy                         0.8166      5304
   macro avg     0.8107    0.8227    0.8129      5304
weighted avg     0.8272    0.8166    0.8183      5304

tensor([0.1362]) ==ece==
===cdia-bias===


100%|██████████| 2829/2829 [00:23<00:00, 120.11it/s]


              precision    recall  f1-score   support

          偏见     0.5761    0.5376    0.5562       718
          正常     0.8462    0.8655    0.8557      2111

    accuracy                         0.7823      2829
   macro avg     0.7112    0.7015    0.7060      2829
weighted avg     0.7777    0.7823    0.7797      2829

tensor([0.0758]) ==ece==
===ciron===


...

              precision    recall  f1-score   support

          正向     0.7700    0.8501    0.8081      1201
          负向     0.8504    0.7703    0.8084      1328

    accuracy                         0.8082      2529
   macro avg     0.8102    0.8102    0.8082      2529
weighted avg     0.8122    0.8082    0.8082      2529

tensor([0.1563]) ==ece==
===senti_smp===


100%|██████████| 2844/2844 [00:23<00:00, 121.86it/s]


              precision    recall  f1-score   support

          正向     0.8057    0.8988    0.8497      1126
          负向     0.9282    0.8580    0.8917      1718

    accuracy                         0.8741      2844
   macro avg     0.8670    0.8784    0.8707      2844
weighted avg     0.8797    0.8741    0.8751      2844

tensor([0.0911]) ==ece==
===senti_query===


 60%|█████▉    | 30108/50509 [04:09<02:48, 120.89it/s]


KeyboardInterrupt: 

In [19]:

evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_raw_filter_senti_query_risk_v4_intent_v2/multitask_cls.pth.8')


===offensive===


100%|██████████| 5304/5304 [00:47<00:00, 111.59it/s]


              precision    recall  f1-score   support

          冒犯     0.7309    0.8680    0.7936      2106
          正常     0.9008    0.7896    0.8415      3198

    accuracy                         0.8207      5304
   macro avg     0.8159    0.8288    0.8176      5304
weighted avg     0.8334    0.8207    0.8225      5304

tensor([0.1185]) ==ece==
===cdia-bias===


100%|██████████| 2829/2829 [00:23<00:00, 122.14it/s]


              precision    recall  f1-score   support

          偏见     0.5839    0.5139    0.5467       718
          正常     0.8411    0.8754    0.8579      2111

    accuracy                         0.7837      2829
   macro avg     0.7125    0.6947    0.7023      2829
weighted avg     0.7758    0.7837    0.7789      2829

tensor([0.0372]) ==ece==
===ciron===


...

              precision    recall  f1-score   support

          正向     0.7702    0.8510    0.8085      1201
          负向     0.8511    0.7703    0.8087      1328

    accuracy                         0.8086      2529
   macro avg     0.8106    0.8106    0.8086      2529
weighted avg     0.8127    0.8086    0.8086      2529

tensor([0.1475]) ==ece==
===senti_smp===


100%|██████████| 2844/2844 [00:23<00:00, 123.29it/s]


              precision    recall  f1-score   support

          正向     0.8225    0.9014    0.8602      1126
          负向     0.9311    0.8725    0.9008      1718

    accuracy                         0.8840      2844
   macro avg     0.8768    0.8870    0.8805      2844
weighted avg     0.8881    0.8840    0.8847      2844

tensor([0.0719]) ==ece==
===senti_query===


 78%|███████▊  | 39435/50509 [05:17<01:29, 124.29it/s]


KeyboardInterrupt: 

In [25]:
evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_query_risk_20221222/multitask_cls.pth.9')


===offensive===


100%|██████████| 5304/5304 [00:43<00:00, 122.71it/s]


              precision    recall  f1-score   support

          冒犯     0.7344    0.8599    0.7922      2106
          正常     0.8961    0.7952    0.8426      3198

    accuracy                         0.8209      5304
   macro avg     0.8152    0.8276    0.8174      5304
weighted avg     0.8319    0.8209    0.8226      5304

tensor([0.1135]) ==ece==
===cdia-bias===


100%|██████████| 2829/2829 [00:23<00:00, 122.61it/s]


              precision    recall  f1-score   support

          偏见     0.5700    0.4930    0.5288       718
          正常     0.8351    0.8735    0.8539      2111

    accuracy                         0.7770      2829
   macro avg     0.7026    0.6833    0.6913      2829
weighted avg     0.7679    0.7770    0.7714      2829

tensor([0.0479]) ==ece==
===ciron===


...

              precision    recall  f1-score   support

          正向     0.7767    0.8543    0.8136      1201
          负向     0.8551    0.7779    0.8147      1328

    accuracy                         0.8142      2529
   macro avg     0.8159    0.8161    0.8142      2529
weighted avg     0.8179    0.8142    0.8142      2529

tensor([0.1410]) ==ece==
===senti_smp===


100%|██████████| 2844/2844 [00:23<00:00, 122.76it/s]


              precision    recall  f1-score   support

          正向     0.8195    0.9032    0.8593      1126
          负向     0.9320    0.8696    0.8997      1718

    accuracy                         0.8829      2844
   macro avg     0.8758    0.8864    0.8795      2844
weighted avg     0.8875    0.8829    0.8837      2844

tensor([0.0736]) ==ece==
===senti_query===


100%|██████████| 50509/50509 [06:45<00:00, 124.52it/s]


              precision    recall  f1-score   support

          正向     0.9686    0.9721    0.9704     28251
          负向     0.9645    0.9600    0.9622     22258

    accuracy                         0.9668     50509
   macro avg     0.9665    0.9661    0.9663     50509
weighted avg     0.9668    0.9668    0.9668     50509

tensor([0.0188]) ==ece==


In [11]:
evaluation('/data/albert.xht/xiaodao/query_risk_v3/multitask_raw_filter_senti_query_risk_v3/multitask_cls.pth.9')

===offensive===


100%|██████████| 5304/5304 [00:42<00:00, 123.49it/s]


              precision    recall  f1-score   support

          冒犯     0.7448    0.8538    0.7956      2106
          正常     0.8934    0.8074    0.8482      3198

    accuracy                         0.8258      5304
   macro avg     0.8191    0.8306    0.8219      5304
weighted avg     0.8344    0.8258    0.8273      5304

tensor([0.1187]) ==ece==
===cdia-bias===


100%|██████████| 2829/2829 [00:22<00:00, 123.55it/s]


              precision    recall  f1-score   support

          偏见     0.5665    0.5752    0.5708       718
          正常     0.8548    0.8503    0.8525      2111

    accuracy                         0.7805      2829
   macro avg     0.7106    0.7128    0.7117      2829
weighted avg     0.7816    0.7805    0.7810      2829

tensor([0.0452]) ==ece==
===ciron===


...

              precision    recall  f1-score   support

          正向     0.9282    0.9595    0.9436       593
          负向     0.9591    0.9275    0.9430       607

    accuracy                         0.9433      1200
   macro avg     0.9437    0.9435    0.9433      1200
weighted avg     0.9438    0.9433    0.9433      1200

tensor([0.0386]) ==ece==
===senti_smpecisa===


100%|██████████| 2529/2529 [00:20<00:00, 124.37it/s]


              precision    recall  f1-score   support

          正向     0.7543    0.8485    0.7986      1201
          负向     0.8455    0.7500    0.7949      1328

    accuracy                         0.7968      2529
   macro avg     0.7999    0.7992    0.7967      2529
weighted avg     0.8022    0.7968    0.7966      2529

tensor([0.1659]) ==ece==
===senti_smp===


100%|██████████| 2844/2844 [00:23<00:00, 123.31it/s]


              precision    recall  f1-score   support

          正向     0.8233    0.8899    0.8553      1126
          负向     0.9238    0.8749    0.8987      1718

    accuracy                         0.8808      2844
   macro avg     0.8736    0.8824    0.8770      2844
weighted avg     0.8840    0.8808    0.8815      2844

tensor([0.0784]) ==ece==


In [156]:

evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_raw_all_ce_256_20/multitask_cls.pth.9')

===offensive===


100%|██████████| 5304/5304 [00:42<00:00, 124.52it/s]


              precision    recall  f1-score   support

          冒犯     0.7084    0.8837    0.7864      2106
          正常     0.9085    0.7605    0.8279      3198

    accuracy                         0.8094      5304
   macro avg     0.8084    0.8221    0.8072      5304
weighted avg     0.8290    0.8094    0.8114      5304

tensor([0.1319]) ==ece==
===cdia-bias===


100%|██████████| 2829/2829 [00:22<00:00, 124.59it/s]


              precision    recall  f1-score   support

          偏见     0.5622    0.6045    0.5826       718
          正常     0.8619    0.8399    0.8508      2111

    accuracy                         0.7801      2829
   macro avg     0.7121    0.7222    0.7167      2829
weighted avg     0.7859    0.7801    0.7827      2829

tensor([0.0458]) ==ece==
===ciron===


...

              precision    recall  f1-score   support

          正向     0.9112    0.9174    0.9143       593
          负向     0.9187    0.9127    0.9157       607

    accuracy                         0.9150      1200
   macro avg     0.9150    0.9150    0.9150      1200
weighted avg     0.9150    0.9150    0.9150      1200

tensor([0.0297]) ==ece==
===senti_smpecisa===


100%|██████████| 2529/2529 [00:20<00:00, 126.30it/s]


              precision    recall  f1-score   support

          正向     0.8326    0.7868    0.8091      1201
          负向     0.8164    0.8569    0.8361      1328

    accuracy                         0.8236      2529
   macro avg     0.8245    0.8219    0.8226      2529
weighted avg     0.8241    0.8236    0.8233      2529

tensor([0.0308]) ==ece==
===senti_smp===


100%|██████████| 2844/2844 [00:22<00:00, 126.33it/s]


              precision    recall  f1-score   support

          正向     0.8724    0.8321    0.8518      1126
          负向     0.8932    0.9203    0.9065      1718

    accuracy                         0.8854      2844
   macro avg     0.8828    0.8762    0.8792      2844
weighted avg     0.8850    0.8854    0.8849      2844

tensor([0.0111]) ==ece==


In [46]:
evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_raw_all_intent_v1/multitask_cls.pth.9')

===offensive===


100%|██████████| 5304/5304 [00:42<00:00, 125.32it/s]


              precision    recall  f1-score   support

          冒犯     0.7050    0.8794    0.7826      2106
          正常     0.9051    0.7577    0.8249      3198

    accuracy                         0.8060      5304
   macro avg     0.8051    0.8185    0.8037      5304
weighted avg     0.8257    0.8060    0.8081      5304

tensor([0.1119]) ==ece==
===cdia-bias===


100%|██████████| 2829/2829 [00:25<00:00, 111.92it/s]


              precision    recall  f1-score   support

          偏见     0.6208    0.4903    0.5479       718
          正常     0.8382    0.8982    0.8671      2111

    accuracy                         0.7946      2829
   macro avg     0.7295    0.6942    0.7075      2829
weighted avg     0.7830    0.7946    0.7861      2829

tensor([0.0231]) ==ece==
===ciron===


...

              precision    recall  f1-score   support

          正向     0.8838    0.9106    0.8970       593
          负向     0.9100    0.8830    0.8963       607

    accuracy                         0.8967      1200
   macro avg     0.8969    0.8968    0.8967      1200
weighted avg     0.8971    0.8967    0.8967      1200

tensor([0.0226]) ==ece==
===senti_smpecisa===


100%|██████████| 2529/2529 [00:22<00:00, 111.83it/s]


              precision    recall  f1-score   support

          正向     0.8053    0.8160    0.8106      1201
          负向     0.8316    0.8215    0.8265      1328

    accuracy                         0.8189      2529
   macro avg     0.8184    0.8188    0.8186      2529
weighted avg     0.8191    0.8189    0.8190      2529

tensor([0.0208]) ==ece==
===senti_smp===


100%|██████████| 2844/2844 [00:22<00:00, 125.04it/s]


              precision    recall  f1-score   support

          正向     0.8540    0.8366    0.8452      1126
          负向     0.8943    0.9063    0.9003      1718

    accuracy                         0.8787      2844
   macro avg     0.8742    0.8714    0.8727      2844
weighted avg     0.8784    0.8787    0.8785      2844

tensor([0.0247]) ==ece==


In [48]:
evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_raw_all_focal/multitask_cls.pth.9')

===offensive===


100%|██████████| 5304/5304 [00:42<00:00, 125.80it/s]


              precision    recall  f1-score   support

          冒犯     0.7068    0.8746    0.7818      2106
          正常     0.9021    0.7611    0.8256      3198

    accuracy                         0.8062      5304
   macro avg     0.8045    0.8179    0.8037      5304
weighted avg     0.8246    0.8062    0.8082      5304

tensor([0.0195]) ==ece==
===cdia-bias===


100%|██████████| 2829/2829 [00:22<00:00, 125.39it/s]


              precision    recall  f1-score   support

          偏见     0.5951    0.4749    0.5283       718
          正常     0.8329    0.8901    0.8605      2111

    accuracy                         0.7847      2829
   macro avg     0.7140    0.6825    0.6944      2829
weighted avg     0.7725    0.7847    0.7762      2829

tensor([0.1170]) ==ece==
===ciron===


...

              precision    recall  f1-score   support

          正向     0.8807    0.9089    0.8946       593
          负向     0.9082    0.8797    0.8937       607

    accuracy                         0.8942      1200
   macro avg     0.8944    0.8943    0.8942      1200
weighted avg     0.8946    0.8942    0.8942      1200

tensor([0.1561]) ==ece==
===senti_smpecisa===


100%|██████████| 2529/2529 [00:20<00:00, 126.43it/s]


              precision    recall  f1-score   support

          正向     0.8163    0.8251    0.8207      1201
          负向     0.8403    0.8321    0.8362      1328

    accuracy                         0.8288      2529
   macro avg     0.8283    0.8286    0.8284      2529
weighted avg     0.8289    0.8288    0.8288      2529

tensor([0.1556]) ==ece==
===senti_smp===


100%|██████████| 2844/2844 [00:22<00:00, 125.37it/s]


              precision    recall  f1-score   support

          正向     0.8627    0.8313    0.8467      1126
          负向     0.8920    0.9133    0.9025      1718

    accuracy                         0.8808      2844
   macro avg     0.8773    0.8723    0.8746      2844
weighted avg     0.8804    0.8808    0.8804      2844

tensor([0.1674]) ==ece==


In [50]:
evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_balanced_intent_v1//multitask_cls.pth.9')



===offensive===


100%|██████████| 5304/5304 [00:42<00:00, 124.95it/s]


              precision    recall  f1-score   support

          冒犯     0.7034    0.8818    0.7826      2106
          正常     0.9065    0.7552    0.8240      3198

    accuracy                         0.8054      5304
   macro avg     0.8050    0.8185    0.8033      5304
weighted avg     0.8259    0.8054    0.8075      5304

tensor([0.1129]) ==ece==
===cdia-bias===


100%|██████████| 2829/2829 [00:22<00:00, 125.58it/s]


              precision    recall  f1-score   support

          偏见     0.6035    0.4833    0.5367       718
          正常     0.8354    0.8920    0.8628      2111

    accuracy                         0.7883      2829
   macro avg     0.7194    0.6876    0.6998      2829
weighted avg     0.7765    0.7883    0.7800      2829

tensor([0.0261]) ==ece==
===ciron===


...

              precision    recall  f1-score   support

          正向     0.8831    0.9174    0.8999       593
          负向     0.9161    0.8814    0.8984       607

    accuracy                         0.8992      1200
   macro avg     0.8996    0.8994    0.8992      1200
weighted avg     0.8998    0.8992    0.8992      1200

tensor([0.0285]) ==ece==
===senti_smpecisa===


100%|██████████| 2529/2529 [00:19<00:00, 127.17it/s]


              precision    recall  f1-score   support

          正向     0.8099    0.8301    0.8199      1201
          负向     0.8428    0.8238    0.8332      1328

    accuracy                         0.8268      2529
   macro avg     0.8264    0.8270    0.8266      2529
weighted avg     0.8272    0.8268    0.8269      2529

tensor([0.0214]) ==ece==
===senti_smp===


100%|██████████| 2844/2844 [00:22<00:00, 126.09it/s]


              precision    recall  f1-score   support

          正向     0.8552    0.8393    0.8472      1126
          负向     0.8959    0.9069    0.9014      1718

    accuracy                         0.8801      2844
   macro avg     0.8756    0.8731    0.8743      2844
weighted avg     0.8798    0.8801    0.8799      2844

tensor([0.0243]) ==ece==


In [51]:
evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_raw_all_focal_128/multitask_cls.pth.9')


===offensive===


100%|██████████| 5304/5304 [00:42<00:00, 125.00it/s]


              precision    recall  f1-score   support

          冒犯     0.7314    0.8661    0.7930      2106
          正常     0.8996    0.7905    0.8415      3198

    accuracy                         0.8205      5304
   macro avg     0.8155    0.8283    0.8173      5304
weighted avg     0.8328    0.8205    0.8223      5304

tensor([0.0103]) ==ece==
===cdia-bias===


100%|██████████| 2829/2829 [00:22<00:00, 124.37it/s]


              precision    recall  f1-score   support

          偏见     0.5877    0.4944    0.5371       718
          正常     0.8369    0.8820    0.8589      2111

    accuracy                         0.7837      2829
   macro avg     0.7123    0.6882    0.6980      2829
weighted avg     0.7736    0.7837    0.7772      2829

tensor([0.1111]) ==ece==
===ciron===


...

              precision    recall  f1-score   support

          正向     0.8941    0.9258    0.9097       593
          负向     0.9249    0.8929    0.9086       607

    accuracy                         0.9092      1200
   macro avg     0.9095    0.9094    0.9092      1200
weighted avg     0.9097    0.9092    0.9092      1200

tensor([0.1246]) ==ece==
===senti_smpecisa===


100%|██████████| 2529/2529 [00:20<00:00, 125.53it/s]


              precision    recall  f1-score   support

          正向     0.8103    0.8218    0.8160      1201
          负向     0.8368    0.8261    0.8314      1328

    accuracy                         0.8240      2529
   macro avg     0.8236    0.8239    0.8237      2529
weighted avg     0.8242    0.8240    0.8241      2529

tensor([0.1440]) ==ece==
===senti_smp===


100%|██████████| 2844/2844 [00:22<00:00, 124.13it/s]


              precision    recall  f1-score   support

          正向     0.8532    0.8464    0.8498      1126
          负向     0.8998    0.9045    0.9022      1718

    accuracy                         0.8815      2844
   macro avg     0.8765    0.8754    0.8760      2844
weighted avg     0.8814    0.8815    0.8814      2844

tensor([0.1548]) ==ece==


In [52]:
evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_contrast_balanced_intent_v1/multitask_cls.pth.9')


===offensive===


100%|██████████| 5304/5304 [00:42<00:00, 125.22it/s]


              precision    recall  f1-score   support

          冒犯     0.7074    0.8770    0.7831      2106
          正常     0.9038    0.7611    0.8263      3198

    accuracy                         0.8071      5304
   macro avg     0.8056    0.8191    0.8047      5304
weighted avg     0.8258    0.8071    0.8092      5304

tensor([0.1762]) ==ece==
===cdia-bias===


100%|██████████| 2829/2829 [00:22<00:00, 124.54it/s]


              precision    recall  f1-score   support

          偏见     0.5777    0.5125    0.5432       718
          正常     0.8403    0.8726    0.8561      2111

    accuracy                         0.7812      2829
   macro avg     0.7090    0.6926    0.6997      2829
weighted avg     0.7737    0.7812    0.7767      2829

tensor([0.1900]) ==ece==
===ciron===


...

              precision    recall  f1-score   support

          正向     0.9314    0.9157    0.9235       593
          负向     0.9190    0.9341    0.9265       607

    accuracy                         0.9250      1200
   macro avg     0.9252    0.9249    0.9250      1200
weighted avg     0.9251    0.9250    0.9250      1200

tensor([0.0596]) ==ece==
===senti_smpecisa===


100%|██████████| 2529/2529 [00:20<00:00, 125.97it/s]


              precision    recall  f1-score   support

          正向     0.8183    0.8285    0.8233      1201
          负向     0.8431    0.8336    0.8383      1328

    accuracy                         0.8312      2529
   macro avg     0.8307    0.8310    0.8308      2529
weighted avg     0.8313    0.8312    0.8312      2529

tensor([0.1288]) ==ece==
===senti_smp===


100%|██████████| 2844/2844 [00:22<00:00, 126.45it/s]


              precision    recall  f1-score   support

          正向     0.8613    0.8437    0.8524      1126
          负向     0.8989    0.9109    0.9049      1718

    accuracy                         0.8843      2844
   macro avg     0.8801    0.8773    0.8786      2844
weighted avg     0.8840    0.8843    0.8841      2844

tensor([0.0493]) ==ece==


In [66]:
evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_mtdnn_ce_256/multitask_cls.pth.9')


===offensive===


100%|██████████| 5304/5304 [00:42<00:00, 125.73it/s]


              precision    recall  f1-score   support

          冒犯     0.7229    0.8756    0.7919      2106
          正常     0.9048    0.7789    0.8372      3198

    accuracy                         0.8173      5304
   macro avg     0.8138    0.8273    0.8145      5304
weighted avg     0.8326    0.8173    0.8192      5304

tensor([0.1323]) ==ece==
===cdia-bias===


100%|██████████| 2829/2829 [00:22<00:00, 126.27it/s]


              precision    recall  f1-score   support

          偏见     0.5892    0.5153    0.5498       718
          正常     0.8419    0.8778    0.8595      2111

    accuracy                         0.7858      2829
   macro avg     0.7155    0.6966    0.7046      2829
weighted avg     0.7778    0.7858    0.7809      2829

tensor([0.0623]) ==ece==
===ciron===


...

              precision    recall  f1-score   support

          正向     0.9129    0.9191    0.9160       593
          负向     0.9204    0.9143    0.9174       607

    accuracy                         0.9167      1200
   macro avg     0.9166    0.9167    0.9167      1200
weighted avg     0.9167    0.9167    0.9167      1200

tensor([0.0421]) ==ece==
===senti_smpecisa===


100%|██████████| 2529/2529 [00:19<00:00, 126.73it/s]


              precision    recall  f1-score   support

          正向     0.8085    0.8085    0.8085      1201
          负向     0.8268    0.8268    0.8268      1328

    accuracy                         0.8181      2529
   macro avg     0.8177    0.8177    0.8177      2529
weighted avg     0.8181    0.8181    0.8181      2529

tensor([0.0537]) ==ece==
===senti_smp===


100%|██████████| 2844/2844 [00:22<00:00, 124.94it/s]


              precision    recall  f1-score   support

          正向     0.8543    0.8330    0.8435      1126
          负向     0.8923    0.9069    0.8995      1718

    accuracy                         0.8776      2844
   macro avg     0.8733    0.8700    0.8715      2844
weighted avg     0.8773    0.8776    0.8774      2844

tensor([0.0230]) ==ece==


In [67]:
evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_mtdnn_focal_256/multitask_cls.pth.9')


===offensive===


100%|██████████| 5304/5304 [00:42<00:00, 125.26it/s]


              precision    recall  f1-score   support

          冒犯     0.7248    0.8590    0.7862      2106
          正常     0.8942    0.7852    0.8362      3198

    accuracy                         0.8145      5304
   macro avg     0.8095    0.8221    0.8112      5304
weighted avg     0.8269    0.8145    0.8163      5304

tensor([0.0393]) ==ece==
===cdia-bias===


100%|██████████| 2829/2829 [00:22<00:00, 124.76it/s]


              precision    recall  f1-score   support

          偏见     0.5753    0.5056    0.5382       718
          正常     0.8385    0.8730    0.8554      2111

    accuracy                         0.7798      2829
   macro avg     0.7069    0.6893    0.6968      2829
weighted avg     0.7717    0.7798    0.7749      2829

tensor([0.0811]) ==ece==
===ciron===


...

              precision    recall  f1-score   support

          正向     0.9132    0.9224    0.9178       593
          负向     0.9235    0.9143    0.9189       607

    accuracy                         0.9183      1200
   macro avg     0.9183    0.9184    0.9183      1200
weighted avg     0.9184    0.9183    0.9183      1200

tensor([0.0882]) ==ece==
===senti_smpecisa===


100%|██████████| 2529/2529 [00:20<00:00, 125.80it/s]


              precision    recall  f1-score   support

          正向     0.8040    0.8027    0.8033      1201
          负向     0.8218    0.8230    0.8224      1328

    accuracy                         0.8134      2529
   macro avg     0.8129    0.8129    0.8129      2529
weighted avg     0.8134    0.8134    0.8134      2529

tensor([0.0975]) ==ece==
===senti_smp===


100%|██████████| 2844/2844 [00:22<00:00, 124.79it/s]


              precision    recall  f1-score   support

          正向     0.8523    0.8250    0.8384      1126
          负向     0.8877    0.9063    0.8969      1718

    accuracy                         0.8741      2844
   macro avg     0.8700    0.8657    0.8677      2844
weighted avg     0.8737    0.8741    0.8738      2844

tensor([0.1215]) ==ece==


In [71]:
evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_raw_all_ce_256/multitask_cls.pth.9')




===offensive===


100%|██████████| 6417/6417 [00:51<00:00, 124.88it/s]


              precision    recall  f1-score   support

          冒犯     0.9025    0.9264    0.9143      3208
          正常     0.9245    0.9000    0.9120      3209

    accuracy                         0.9132      6417
   macro avg     0.9135    0.9132    0.9132      6417
weighted avg     0.9135    0.9132    0.9132      6417

tensor([0.0343]) ==ece==
===cdia-bias===


100%|██████████| 2829/2829 [00:22<00:00, 124.98it/s]


              precision    recall  f1-score   support

          偏见     0.5808    0.5153    0.5461       718
          正常     0.8412    0.8735    0.8571      2111

    accuracy                         0.7826      2829
   macro avg     0.7110    0.6944    0.7016      2829
weighted avg     0.7752    0.7826    0.7782      2829

tensor([0.0418]) ==ece==
===ciron===


...

              precision    recall  f1-score   support

          正向     0.9028    0.9241    0.9133       593
          负向     0.9241    0.9028    0.9133       607

    accuracy                         0.9133      1200
   macro avg     0.9135    0.9135    0.9133      1200
weighted avg     0.9136    0.9133    0.9133      1200

tensor([0.0361]) ==ece==
===senti_smpecisa===


100%|██████████| 2529/2529 [00:19<00:00, 126.84it/s]


              precision    recall  f1-score   support

          正向     0.8196    0.8285    0.8240      1201
          负向     0.8433    0.8351    0.8392      1328

    accuracy                         0.8319      2529
   macro avg     0.8315    0.8318    0.8316      2529
weighted avg     0.8321    0.8319    0.8320      2529

tensor([0.0217]) ==ece==
===senti_smp===


100%|██████████| 2844/2844 [00:22<00:00, 126.04it/s]


              precision    recall  f1-score   support

          正向     0.8630    0.8446    0.8537      1126
          负向     0.8995    0.9121    0.9058      1718

    accuracy                         0.8854      2844
   macro avg     0.8813    0.8783    0.8797      2844
weighted avg     0.8851    0.8854    0.8852      2844

tensor([0.0128]) ==ece==


In [72]:
evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_raw_all_focal_256/multitask_cls.pth.9')


===offensive===


100%|██████████| 6417/6417 [00:51<00:00, 125.02it/s]


              precision    recall  f1-score   support

          冒犯     0.9075    0.9261    0.9167      3208
          正常     0.9246    0.9056    0.9150      3209

    accuracy                         0.9158      6417
   macro avg     0.9160    0.9159    0.9158      6417
weighted avg     0.9160    0.9158    0.9158      6417

tensor([0.0770]) ==ece==
===cdia-bias===


100%|██████████| 2829/2829 [00:25<00:00, 109.98it/s]


              precision    recall  f1-score   support

          偏见     0.5963    0.4958    0.5414       718
          正常     0.8378    0.8858    0.8612      2111

    accuracy                         0.7869      2829
   macro avg     0.7171    0.6908    0.7013      2829
weighted avg     0.7765    0.7869    0.7800      2829

tensor([0.1125]) ==ece==
===ciron===


...

              precision    recall  f1-score   support

          正向     0.8992    0.9174    0.9082       593
          负向     0.9176    0.8995    0.9085       607

    accuracy                         0.9083      1200
   macro avg     0.9084    0.9084    0.9083      1200
weighted avg     0.9085    0.9083    0.9083      1200

tensor([0.1026]) ==ece==
===senti_smpecisa===


100%|██████████| 2529/2529 [00:19<00:00, 127.02it/s]


              precision    recall  f1-score   support

          正向     0.8167    0.8127    0.8147      1201
          负向     0.8313    0.8351    0.8332      1328

    accuracy                         0.8244      2529
   macro avg     0.8240    0.8239    0.8239      2529
weighted avg     0.8244    0.8244    0.8244      2529

tensor([0.1432]) ==ece==
===senti_smp===


100%|██████████| 2844/2844 [00:22<00:00, 126.06it/s]


              precision    recall  f1-score   support

          正向     0.8662    0.8393    0.8525      1126
          负向     0.8967    0.9150    0.9058      1718

    accuracy                         0.8850      2844
   macro avg     0.8815    0.8771    0.8791      2844
weighted avg     0.8846    0.8850    0.8847      2844

tensor([0.1520]) ==ece==


In [75]:
evaluation('/data/albert.xht/xiaodao/risk_classification/multitask_contrast_intent_v1/multitask_cls.pth.9')


===offensive===


100%|██████████| 5304/5304 [00:42<00:00, 125.36it/s]


              precision    recall  f1-score   support

          冒犯     0.7202    0.8371    0.7743      2106
          正常     0.8799    0.7858    0.8302      3198

    accuracy                         0.8062      5304
   macro avg     0.8000    0.8115    0.8022      5304
weighted avg     0.8165    0.8062    0.8080      5304

tensor([0.0912]) ==ece==
===cdia-bias===


100%|██████████| 2829/2829 [00:22<00:00, 125.30it/s]


              precision    recall  f1-score   support

          偏见     0.6115    0.3245    0.4240       718
          正常     0.8019    0.9299    0.8612      2111

    accuracy                         0.7762      2829
   macro avg     0.7067    0.6272    0.6426      2829
weighted avg     0.7536    0.7762    0.7502      2829

tensor([0.0236]) ==ece==
===ciron===


...

              precision    recall  f1-score   support

          正向     0.8849    0.8820    0.8834       593
          负向     0.8851    0.8880    0.8865       607

    accuracy                         0.8850      1200
   macro avg     0.8850    0.8850    0.8850      1200
weighted avg     0.8850    0.8850    0.8850      1200

tensor([0.0228]) ==ece==
===senti_smpecisa===


100%|██████████| 2529/2529 [00:19<00:00, 126.66it/s]


              precision    recall  f1-score   support

          正向     0.7964    0.8010    0.7987      1201
          负向     0.8191    0.8148    0.8169      1328

    accuracy                         0.8082      2529
   macro avg     0.8077    0.8079    0.8078      2529
weighted avg     0.8083    0.8082    0.8082      2529

tensor([0.0255]) ==ece==
===senti_smp===


100%|██████████| 2844/2844 [00:22<00:00, 126.16it/s]


              precision    recall  f1-score   support

          正向     0.8475    0.8144    0.8306      1126
          负向     0.8814    0.9040    0.8925      1718

    accuracy                         0.8685      2844
   macro avg     0.8644    0.8592    0.8616      2844
weighted avg     0.8680    0.8685    0.8680      2844

tensor([0.0368]) ==ece==


In [24]:
ece_fn = ECE(n_bins=15)

In [45]:
mapping_dict = {
    '冒犯':0,
    '正常':1
}

gold_l = torch.tensor([mapping_dict[item] for item in gold])
pred_score_l = torch.tensor([[item[0][1], item[1][1]] for item in pred_score])

ece_fn(pred_score_l, gold_l, mode='probs')

tensor([0.1104])

In [43]:
mapping_dict = {
    '负向':0,
    '正向':1
}

gold_l = torch.tensor([mapping_dict[item] for item in gold])
pred_score_l = torch.tensor([[item[0][1], item[1][1]] for item in pred_score])

ece_fn(pred_score_l, gold_l, mode='probs')

tensor([0.0198])